In [1]:
!pip install sentencepiece
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install torch
!pip install sentencepiece
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292695 sha256=516e88c3d2d992667fca628e5ff59b121e4ebadb1104411aa6349b39c8294b9b
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp


In [2]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-aseoh7mg/kobert-tokenizer_7b26de9ffdec454f8ec73d77dcabbe59
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-aseoh7mg/kobert-tokenizer_7b26de9ffdec454f8ec73d77dcabbe59
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=70b18b101ae499af42bfb48d74cc25a3bac0b3b72336c7b36b55193010d9c174
  Stored in directory: /tmp/pip-ephem-wheel-cache-hk7bqcj4/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
from PIL import Image
import numpy as np
import os
import glob

In [5]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [6]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import glob
import os

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [8]:
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)

각 폴더에서 이미지와 텍스트 저장한 폴더 위치

In [9]:
section_folders = glob.glob('/content/drive/MyDrive/NaverNews/*')
section_folders

['/content/drive/MyDrive/NaverNews/politics',
 '/content/drive/MyDrive/NaverNews/society',
 '/content/drive/MyDrive/NaverNews/science',
 '/content/drive/MyDrive/NaverNews/life',
 '/content/drive/MyDrive/NaverNews/world',
 '/content/drive/MyDrive/NaverNews/economy']

In [10]:
root = '/content/drive/MyDrive/NaverNews'
sections = os.listdir(root)
sections

['politics', 'society', 'science', 'life', 'world', 'economy']

텍스트 위치

In [11]:
text_paths = []
for folder, section in zip(section_folders, sections):
  text_path = folder + '/' + section + 'text' + '.xlsx'
  text_paths.append(text_path)

text_paths

['/content/drive/MyDrive/NaverNews/politics/politicstext.xlsx',
 '/content/drive/MyDrive/NaverNews/society/societytext.xlsx',
 '/content/drive/MyDrive/NaverNews/science/sciencetext.xlsx',
 '/content/drive/MyDrive/NaverNews/life/lifetext.xlsx',
 '/content/drive/MyDrive/NaverNews/world/worldtext.xlsx',
 '/content/drive/MyDrive/NaverNews/economy/economytext.xlsx']

각 섹션별 인덱스 딕셔너리

In [12]:
# 각 섹션별 사용할 데이터의 인덱스가 담겨 있는 딕셔너리

idx_dictionary = {
    'politics' : range(200),
    'society' : range(200),
    'science' : range(200),
    'life' : range(200),
    'world' : range(200),
    'economy' : range(200)
}

label - y 페어

In [13]:
# label_to_y = {section : idx for idx, section in enumerate(sections)}
# label_to_y

In [14]:
label_to_y = {
    'politics': 0,
    'society': 1,
    'science': 2,
    'life': 3,
    'world': 4,
    'economy': 5
}
label_to_y

{'politics': 0,
 'society': 1,
 'science': 2,
 'life': 3,
 'world': 4,
 'economy': 5}

분석에 사용할 이미지 데이터 경로

In [82]:
# 분석에 사용할 모든 데이터들의 경로를 불러온다.

overall_imgpaths = []

for section_folder in section_folders:

  # 각 섹션 이름 가져와서 인덱스랑 합하기
  section = os.path.basename(section_folder)

  section_imgpaths = []

  for idx in idx_dictionary[section]:
    imgname = section + str(idx) + '.jpg'
    imgpath = os.path.join(section_folder, imgname)

    section_imgpaths.append(imgpath)

  overall_imgpaths.extend(section_imgpaths)

overall_imgpaths[:10]

['/content/drive/MyDrive/NaverNews/politics/politics0.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics1.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics2.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics3.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics4.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics5.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics6.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics7.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics8.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics9.jpg']

분석에 사용할 텍스트 데이터 DF

In [83]:
overall_text = []

for section_folder in section_folders:
  section = os.path.basename(section_folder)
  section_textpath = section_folder + '/' + section + 'text' + '.xlsx'

  section_texts_raw = pd.read_excel(section_textpath)


  idx = idx_dictionary[section]

  # X
  section_texts = section_texts_raw.iloc[idx]
  annotations = section_texts['annotation']
  articles = section_texts['title'] + section_texts['body']

  # y
  y = label_to_y[section]

  for annotation, article in zip(annotations, articles):
    data = []
    data.append(annotation)
    data.append(article)
    data.append(y)

    overall_text.append(data)

In [17]:
overall_text

[['축사하는 캠벨 미 백악관 NSC 조정관[연합뉴스 자료사진]',
  '美국무부 부장관에 \'한미동맹 중시\' 캠벨 백악관 조정관 지명돼(종합2보)(워싱턴=연합뉴스) 김동현 특파원 = 미국 국무부 2인자인 부장관에 한미동맹의 가치를 높게 평가하는 아시아 전문가가 지명됐다.      조 바이든 미국 대통령이 커트 캠벨 백악관 국가안전보장회의(NSC) 인도·태평양 조정관을 차기 국무부 부장관으로 지명할 방침이라고 백악관이 1일(현지시간) 밝혔다.    캠벨 조정관은 2021년 1월 20일부터 인태조정관으로 재직하면서 북한을 비롯한 한반도 문제와 미국의 아시아 정책을 총괄해왔다.    캠벨 조정관은 바이든 행정부가 대표적 외교 성과로 꼽는 한미일 관계 개선에 깊이 관여하고 한미 핵협의그룹(NCG) 첫 회의에서 미국 대표를 맡는 등 한미 관계에 정통하고 동맹의 중요성에 누구보다 무게를 두는 핵심 인사로 꼽힌다.    버락 오바마 행정부 시절인 2009∼2013년 국무부 동아태 차관보를 지내며 미국 외교의 초점을 인도·태평양으로 전환한 \'아시아 중심 정책\'(Pivot to Asia)\'을 설계했다.    대학 시절 소련을 전공했고 러시아어도 하지만, 아시아 업무만 거의 40년을 하면서 \'아시아 차르\'로 불리게 됐다.     워싱턴포스트(WP)는 캠벨이 부장관이 되면 미국이 과거 이슬람 무장세력 \'이슬람국가\'(IS)와 전쟁 때처럼 이스라엘과 하마스 분쟁에 집중하느라 인도·태평양 지역을 소홀히 할 수 있다는 아시아 동맹들의 우려를 완화할 수 있다는 의견을 제시했다.     빅터 차 전략국제문제연구소(CSIS) 한국 석좌는 WP에 "아시아 동맹들은 여전히 걱정하겠지만 캠벨을 국무부 2인자로 두면 바이든 행정부가 인도·태평양 전략을 진전시키고 이행할 의지가 있다는 어느 정도의 확신을 줄 것"이라고 말했다.WP에 따르면 캠벨은 지난달 30일 워싱턴DC의 싱크탱크 만찬에서 부장관으로 인준되면 인도·태평양의 동맹과 파트너와 협력관계 심화를 계속 추진할 것이라고 밝혔다.

In [84]:
overall_data = pd.DataFrame(overall_text)
overall_data.columns = ['annotation', 'article', 'y']

In [85]:
overall_data['imgpath'] = overall_imgpaths
overall_data

,annotation,article,y,imgpath
0,축사하는 캠벨 미 백악관 NSC 조정관[연합뉴스 자료사진],美국무부 부장관에 '한미동맹 중시' 캠벨 백악관 조정관 지명돼(종합2보)(워싱턴=연...,0,/content/drive/MyDrive/NaverNews/politics/poli...
1,"""모든건 제 책임"" 자세 낮춰""비상한 각오로 민생 챙길것""은행·카카오택시 횡포 직격","尹, 민생 타운홀 미팅 …""정치과잉 희생자는 서민""◆ 비상경제민생회의 ◆윤석열 대통...",0,/content/drive/MyDrive/NaverNews/politics/poli...
2,김건희 여사가 1일 서울 강남구 코엑스에서 열린 디자인코리아 2023 개막식을 찾아...,"김건희 여사 ""미래는 예술과 디자인에 답 있다""…'깜짝' 질문도김건희 여사는 1일 ...",0,/content/drive/MyDrive/NaverNews/politics/poli...
3,축사하는 캠벨 NSC 조정관[연합뉴스 자료사진],"바이든, 미 국무부 부장관에 캠벨 백악관 인태조정관 지명(종합)(워싱턴=연합뉴스) ...",0,/content/drive/MyDrive/NaverNews/politics/poli...
4,김건희 여사가 1일 서울 강남구 코엑스에서 열린 디자인코리아 2023 개막식에서 축...,"디자인에 진심인 김건희, 직접 축사 작성..축사 도중 즉석 질문도 [파이낸셜뉴스] ...",0,/content/drive/MyDrive/NaverNews/politics/poli...
...,...,...,...,...
1195,[사진 출처 = 국가통계포털],“서울 집값 실화냐”…주택수요 직주근접 ‘젊은 도시’로 이동서울지역 아파트 분양가격...,5,/content/drive/MyDrive/NaverNews/economy/econo...
1196,용두1-6 지구 위치도./서울시 제공,청량리역 근처 1000세대 아파트 공급 지하철 1호선과 수인분당선이 지...,5,/content/drive/MyDrive/NaverNews/economy/econo...
1197,2일 서울 구로구 한 아파트에서 열린 한미글로벌·따뜻한동행 공간복지 지원사업 100...,"한미글로벌, 장애인 공간복지 '1000호 달성' 한미글로벌과 사회복지법인 따뜻한동행...",5,/content/drive/MyDrive/NaverNews/economy/econo...
1198,반포 아크로리버파크 전경. 사진 연합뉴스,`매물 홍수`도 비껴간 초고가 아파트… 반포 아리팍 110억 최고가서초구 반포 아파...,5,/content/drive/MyDrive/NaverNews/economy/econo...


In [86]:
# MobileNet image transform

train_imgtransform = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.Resize(256),
    #transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_imgtransform = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.Resize(256),
    #transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [87]:
# BERT transform

class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [88]:
class RoBaMFEnsembleDataset(Dataset):
  def __init__(self, dataset, imgtransform, bert_tokenizer, vocab, max_len_article = 100, max_len_annotation = 30, pad = None, pair = None):

    # for ImageBaseline
    self.imgpaths = dataset['imgpath']
    self.imgtransform = imgtransform


    # for ArticleBaseline & FusionBaseline
    articletransform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len_article, vocab=vocab, pad=pad, pair=pair)
    annotationtransform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len_annotation, vocab=vocab, pad=pad, pair=pair)

    self.kobertdata = dataset[['annotation', 'article', 'y']].values.tolist()
    self.annotations = [annotationtransform([i[0]]) for i in self.kobertdata]
    self.articles = [articletransform([i[1]]) for i in self.kobertdata]


    self.labels = [np.int32(i[2]) for i in self.kobertdata]


  def __getitem__(self, i):
    imgpath = self.imgpaths.iloc[i]

    img = Image.open(imgpath).convert('RGB')
    img = self.imgtransform(img)

    article = self.articles[i]
    annotation = self.annotations[i]

    target = self.labels[i]

    ## article & annotation : (token_ids, valid_length, segment_ids) tuple
    return img, article, annotation, target

  def __len__(self):
    return len(self.labels)


In [89]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [90]:
# kobert 공식 git에 있는 get_kobert_model 선언
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj


In [91]:
from transformers import BertModel
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1', tokenizer.vocab_file)
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

코버트 특징추출기 및 Classifier

In [92]:
class BERTFeatureExtractor(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = None,   # Feature length : 1024
                 dr_rate = None,
                 params = None):
        super(BERTFeatureExtractor, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
          out = self.dropout(pooler)
        #else:
        #  out = pooler
        return self.classifier(out)

피쳐퓨전 Classifier

In [108]:
class FusionModel(nn.Module):
  def __init__(self, mobilenetv2, kobert):
    super(FusionModel, self).__init__()

    self.mobilenetv2 = mobilenetv2
    self.kobert = kobert

    self.fc_image = nn.Linear(1000, 512)
    self.fc_text = nn.Linear(1024, 512)

    self.classifier = nn.Sequential(
        nn.Linear(1024, 1024),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size = 2, stride = 2, padding = 0),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size = 2, stride = 2, padding = 0),
        nn.Linear(256, 6),
        nn.Softmax(1)
    )

  def forward(self, img, token_ids, valid_length, segment_ids):
        # 이미지 특징 추출
        image_feature = self.mobilenetv2(img)
        image_feature = self.fc_image(image_feature)

        # 텍스트 특징 추출
        text_feature = self.kobert(token_ids, valid_length, segment_ids)
        text_feature = self.fc_text(text_feature)

        # 두 특징을 결합
        x = torch.cat((image_feature, text_feature), dim=1)

        # 분류기 적용
        x = self.classifier(x)

        return x

Image Baseline

In [109]:
class ImageModel(nn.Module):
  def __init__(self, mobilenetv2):
    super(ImageModel, self).__init__()

    self.mobilenetv2 = mobilenetv2

    self.fc = nn.Sequential(
              nn.Linear(1000, 1024),
              nn.ReLU(),
              nn.MaxPool1d(kernel_size=2, stride=2, padding=0),
              nn.Linear(512, 512),
              nn.ReLU(),
              nn.MaxPool1d(kernel_size=2, stride=2, padding=0),
              nn.Linear(256, 6),
              nn.Softmax(1)
              )


  def forward(self, img):
    x = self.mobilenetv2(img)
    x = self.fc(x)

    return x

5-fold CV Stacking

In [110]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits = 5)

In [111]:
list_train_idx = []
list_test_idx = []

for train_idx, test_idx in cv.split(overall_data[['article', 'imgpath']], overall_data['y']):
  list_train_idx.append(train_idx)
  list_test_idx.append(test_idx)

In [112]:
mobilenetv2 = models.mobilenet_v2(weights = 'DEFAULT')

In [113]:
mobilenetv2 = models.mobilenet_v2(weights = 'DEFAULT')
bertmodel_for_article, vocab = get_kobert_model('skt/kobert-base-v1', tokenizer.vocab_file)
bertmodel_for_fusion, vocab = get_kobert_model('skt/kobert-base-v1', tokenizer.vocab_file)

ImageBaseline = ImageModel(mobilenetv2)
ArticleBaseline = BERTFeatureExtractor(bertmodel_for_article, num_classes = 6, dr_rate = 0.5)
kobert_for_fusion = BERTFeatureExtractor(bertmodel_for_fusion, num_classes = 1024, dr_rate = 0.5)
FusionBaseline = FusionModel(mobilenetv2, kobert_for_fusion)

In [114]:
class getBaselineProbs():

  def __init__(self, dataloader):
    self.dataloader = dataloader


  def getImageBaselineProbs(self, ImageBaseline):
    TotalImageProbs = []
    self.ImageBaseline = ImageBaseline
    self.ImageBaseline.eval()

    for img, (article_token_ids, article_valid_length, article_segment_ids), (annotation_token_ids, annotation_valid_length, annotation_segment_ids), label in self.dataloader:

      img = img.to(device)

      BatchImageProbs = ImageBaseline(img)
     #BatchImageProbs = [int(i) for i in BatchImageProbs]
      BatchImageProbs = BatchImageProbs.cpu().detach().numpy()

      TotalImageProbs.extend(BatchImageProbs)
      #TotalImageProbs = pd.DataFrame(TotalImageProbs)

    return TotalImageProbs



  def getArticleBaselineProbs(self, ArticleBaseline):
    TotalArticleProbs = []
    self.ArticleBaseline = ArticleBaseline
    self.ArticleBaseline.eval()

    for img, (article_token_ids, article_valid_length, article_segment_ids), (annotation_token_ids, annotation_valid_length, annotation_segment_ids), label in self.dataloader:

      article_token_ids = article_token_ids.long().to(device)
      article_segment_ids = article_segment_ids.long().to(device)
      article_valid_length = article_valid_length

      BatchArticleProbs = ArticleBaseline(article_token_ids, article_valid_length, article_segment_ids)
     #BatchArticleProbs = [int(i) for i in BatchArticleProbs]
      BatchArticleProbs = BatchArticleProbs.cpu().detach().numpy()

      TotalArticleProbs.extend(BatchArticleProbs)
      #TotalArticleProbs = pd.DataFrame(TotalArticleProbs)

    return TotalArticleProbs


  def getFusionBaselineProbs(self, FusionBaseline):
    TotalFusionProbs = []
    self.FusionBaseline = FusionBaseline
    self.FusionBaseline.eval()

    for img, (article_token_ids, article_valid_length, article_segment_ids), (annotation_token_ids, annotation_valid_length, annotation_segment_ids), label in self.dataloader:

      img = img.to(device)

      annotation_token_ids = annotation_token_ids.long().to(device)
      annotation_segment_ids = annotation_segment_ids.long().to(device)
      annotation_valid_length = annotation_valid_length

      BatchFusionProbs = FusionBaseline(img, annotation_token_ids, annotation_valid_length, annotation_segment_ids)
     #BatchFusionProbs = [float(i) for i in BatchFusionProbs]
      BatchFusionProbs = BatchFusionProbs.cpu().detach().numpy()

      TotalFusionProbs.extend(BatchFusionProbs)
      #TotalFusionProbs = pd.DataFrame(TotalFusionProbs)

    return TotalFusionProbs

In [115]:
root = '/content/drive/MyDrive/Colab Notebooks/Machine Learning - Graduate/Model Weights/'

list_ImageWeightPath = []
list_ArticleWeightPath = []
list_FusionWeightPath = []


for fold in range(1, 6):
  ImageWeightPath = root + 'ImageWeight' + str(fold) + '.pth'
  ArticleWeightPath = root + 'ArticleWeight' + str(fold) + '.pth'
  FusionWeightPath = root + 'FusionWeight' + str(fold) + '.pth'

  list_ImageWeightPath.append(ImageWeightPath)
  list_ArticleWeightPath.append(ArticleWeightPath)
  list_FusionWeightPath.append(FusionWeightPath)

In [116]:
new_dataset = RoBaMFEnsembleDataset(test_data, imgtransform = test_imgtransform, bert_tokenizer = tokenizer, vocab = vocab, max_len_article = 100, max_len_annotation = 30, pad = True, pair = False)  ## overall_data -> new_data
new_dataloader = DataLoader(new_dataset, batch_size = 8, shuffle = True)


ImageProbs_train = pd.DataFrame()
ArticleProbs_train = pd.DataFrame()
FusionProbs_train = pd.DataFrame()
fold = 0

for train_idx, test_idx, ImageWeightPath, ArticleWeightPath, FusionWeightPath in zip(list_train_idx, list_test_idx, list_ImageWeightPath, list_ArticleWeightPath, list_FusionWeightPath):

  fold += 1

  warmup_ratio = 0.1
  num_epochs = 10
  max_grad_norm = 1
  log_interval = 200
  learning_rate =  5e-5


  # 5-fold CV Stacking을 위한 기존 train-test 데이터 셋

  train_data = overall_data.iloc[train_idx]
  test_data = overall_data.iloc[test_idx]

  train_dataset = RoBaMFEnsembleDataset(train_data, imgtransform = train_imgtransform, bert_tokenizer = tokenizer , vocab = vocab, max_len_article = 100, max_len_annotation = 30, pad = True, pair = False)
  test_dataset = RoBaMFEnsembleDataset(test_data, imgtransform = test_imgtransform, bert_tokenizer = tokenizer , vocab = vocab, max_len_article = 100, max_len_annotation = 30, pad = True, pair = False)

  train_dataloader = DataLoader(train_dataset, batch_size = 8, shuffle = True)
  test_dataloader = DataLoader(test_dataset, batch_size = 8, shuffle = True)


  # 모형 초기화

  mobilenetv2 = models.mobilenet_v2(weights = 'DEFAULT')
  bertmodel_for_article, vocab = get_kobert_model('skt/kobert-base-v1', tokenizer.vocab_file)
  bertmodel_for_fusion, vocab = get_kobert_model('skt/kobert-base-v1', tokenizer.vocab_file)

  ImageBaseline = ImageModel(mobilenetv2).to(device)
  ArticleBaseline = BERTFeatureExtractor(bertmodel_for_article, num_classes = 6, dr_rate = 0.5).to(device)
  kobert_for_fusion = BERTFeatureExtractor(bertmodel_for_fusion, num_classes = 1024, dr_rate = 0.5).to(device)
  FusionBaseline = FusionModel(mobilenetv2, kobert_for_fusion).to(device)

  ImageBaseline.load_state_dict(torch.load(ImageWeightPath, map_location = device))
  ArticleBaseline.load_state_dict(torch.load(ArticleWeightPath, map_location=device))
  FusionBaseline.load_state_dict(torch.load(FusionWeightPath, map_location=device))


  # Meta Learner에서 훈련 데이터로 쓰일 Meta Feature (test 데이터의 학습 결과)
  batch_ImageProbs_train = pd.DataFrame(getBaselineProbs(test_dataloader).getImageBaselineProbs(ImageBaseline))
  batch_ArticleProbs_train = pd.DataFrame(getBaselineProbs(test_dataloader).getArticleBaselineProbs(ArticleBaseline))
  batch_FusionProbs_train = pd.DataFrame(getBaselineProbs(test_dataloader).getFusionBaselineProbs(FusionBaseline))

  # Meta Learner에서 시험 데이터로 쓰일 Meta Feature (new 데이터의 학습 결과)
  batch_ImageProbs_test = pd.DataFrame(getBaselineProbs(test_dataloader).getImageBaselineProbs(ImageBaseline))
  batch_ArticleProbs_test = pd.DataFrame(getBaselineProbs(test_dataloader).getArticleBaselineProbs(ArticleBaseline))
  batch_FusionProbs_test = pd.DataFrame(getBaselineProbs(test_dataloader).getFusionBaselineProbs(FusionBaseline))


  # 폴드별 MetaFeatures_train 이어붙이기 (각 폴드별 test_data에 대한 예측 이어붙이기)
  ImageProbs_train = pd.concat([ImageProbs_train, batch_ImageProbs_train])
  ArticleProbs_train = pd.concat([ArticleProbs_train, batch_ArticleProbs_train])
  FusionProbs_train = pd.concat([FusionProbs_train, batch_FusionProbs_train])


  # 폴드별 MetaFeatures_test의 합 (new_data에 대한 각 폴드의 예측의 합)
  if fold == 1:
    ImageProbs_test = batch_ImageProbs_test
    ArticleProbs_test = batch_ArticleProbs_test
    FusionProbs_test = batch_FusionProbs_test

  else:
    ImageProbs_test += batch_ImageProbs_test
    ArticleProbs_test += batch_ArticleProbs_test
    FusionProbs_test += batch_FusionProbs_test

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [119]:
ImageProbs_train

,0,1,2,3,4,5
0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...
235,0.0,0.0,0.0,0.0,1.0,0.0
236,0.0,0.0,0.0,0.0,1.0,0.0
237,0.0,0.0,0.0,0.0,1.0,0.0
238,0.0,0.0,0.0,0.0,1.0,0.0


In [118]:
FusionProbs_train

,0,1,2,3,4,5
0,0.175838,0.160050,0.167316,0.152337,0.195703,0.148756
1,0.163652,0.161308,0.178112,0.161902,0.181710,0.153315
2,0.167491,0.162563,0.175082,0.154922,0.183327,0.156615
3,0.162228,0.159321,0.190496,0.164233,0.169492,0.154230
4,0.159401,0.160413,0.170494,0.169978,0.179689,0.160024
...,...,...,...,...,...,...
235,0.170325,0.170563,0.168813,0.146546,0.165856,0.177898
236,0.162743,0.163497,0.179509,0.149250,0.169092,0.175909
237,0.173184,0.159087,0.184715,0.151163,0.170725,0.161126
238,0.159167,0.184188,0.171412,0.149774,0.158185,0.177275


In [105]:
FusionProbs_test

,0,1,2,3,4,5
0,0.199760,0.125555,-0.187517,-0.372813,0.198562,-0.309045
1,0.126094,0.072992,-0.266587,-0.259212,0.119697,-0.176083
2,0.052186,-0.039656,-0.014157,-0.281526,0.146393,-0.318259
3,0.099264,0.193904,-0.086354,-0.408090,0.294636,-0.309957
4,0.106050,0.196105,-0.061699,-0.110672,0.201417,-0.112480
...,...,...,...,...,...,...
235,0.164326,0.085915,-0.055971,-0.413313,0.182469,-0.261013
236,0.248437,-0.004688,-0.062181,-0.250390,0.144349,-0.227746
237,0.221883,0.060951,0.000506,-0.264574,0.045395,-0.272225
238,0.203663,0.175345,-0.252282,-0.195899,0.129187,-0.229035
